#### **Import Libraries and Configuration**

In [1]:
# Import required functions and classes
from sahi import AutoDetectionModel
from sahi.utils.cv import read_image, read_image_as_pil
from sahi.utils.file import Path, increment_path, list_files, save_json, save_pickle, download_from_url
from sahi.predict import get_prediction, get_sliced_prediction, predict, agg_prediction, get_prediction_batched, get_sliced_prediction_batched, predict 
from sahi.prediction import visualize_object_predictions
from IPython.display import Image
from numpy import asarray
import cv2
import os
import time
from PIL import Image
import json
import numpy as np
from tqdm import tqdm
import time
from sahi.prediction import ObjectPrediction, PredictionResult
from pathlib import Path
from multiprocessing import Pool, cpu_count
import torch
from torchvision.ops import nms, clip_boxes_to_image

/mmfs1/cm/shared/apps_local/python/3.11/envs/torch11.8/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download YOLOv8-S model to 'models/yolov8s.pt'
yolov8_model_path = 'models/yolov8/last.pt'
#download_yolov8s_model(destination_path=yolov8_model_path)

In [3]:
detection_model = AutoDetectionModel.from_pretrained(
    model_type='yolov8',
    model_path=yolov8_model_path,
    confidence_threshold=0.3,
    device="cuda:0", # or 'cpu'
)

In [4]:
def get_slice_parameters(object_density, slice_size):
    
    #start_time = time.time()
    #image_path = "test_data/0000006_06773_d_0000018.jpg"
    #image = Image.open(image_path).convert("RGB")
    #image_width, image_height  = image.size
    #print("Image Width:", image_width)
    #print("Image Height:", image_height)
    #min_dim = min(image_width, image_height)
    #slice_size = min_dim // 4 if min_dim > 1600 else min_dim // 2
    #print(f"Dimension calculation time taken: {(time.time() - start_time)*1000:.2f} ms")

    
    if object_density >= 50:
        #slice_size = min_dim // 4
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.5
        overlap_height_ratio = 0.5
    elif 25 <= object_density < 50:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.25
        overlap_height_ratio = 0.25
    elif 10 <= object_density < 25:
        #slice_size = min_dim // 2
        slice_width = slice_size
        slice_height = slice_size
        overlap_width_ratio = 0.15
        overlap_height_ratio = 0.15
    else:
        return

    return slice_width, slice_height, overlap_width_ratio, overlap_height_ratio

In [26]:
def nms_merge(predictions, iou_threshold=0.5):
    if not predictions:
        return []
    print("IOU Threshold: ", iou_threshold)
    boxes = torch.tensor([p.bbox.to_xyxy() for p in predictions])
    scores = torch.tensor([p.score.value for p in predictions])
    keep_indices = nms(boxes, scores, iou_threshold).tolist()
    #keep_indices = clip_boxes_to_image(boxes, 32).tolist()
    return [predictions[i] for i in keep_indices]

In [28]:
def predict_fine_sliced_images(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    data = {}
    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)

    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")

    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        img_id = next((img["id"] for img in data.get("images", []) if img["file_name"].startswith(filename_wo_ext)), None)

        print("*****************************************")
        print("File Name", filename_wo_ext)
        
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)
                # Base prediction on the sub-image
                base_pred = get_prediction(sub_img, detection_model)
                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)
                
                slice_params = get_slice_parameters(object_density, base_slice_size)

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params

                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)
                    
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    preds = sliced_pred.object_prediction_list
                    print("Sliced Level Prediction Count: ", len(preds))
                else:
                    preds = base_pred.object_prediction_list
                    print("Sliced Level Prediction Count: ", len(preds))

                all_object_predictions = preds
                # Offset predictions to original image coordinates
                #for pred in preds:
                 #   pred.shift_amount(x_offset=x1, y_offset=y1)
                 #   all_object_predictions.append(pred)

        # Apply NMS to merged predictions
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.3)

        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        # COCO conversion
        #coco_preds = [p.to_coco_predictions(image_id=img_id) for p in merged_preds]
        #all_coco_preds.extend(coco_preds)

    #if dataset_json_path:
     #   save_json(all_coco_preds, str(save_dir / "result.json"))
      #  print(f"\n Saved COCO results to {save_dir / 'result.json'}")

    print(f"\n✅ Completed {len(image_files)} images.")
    print(f"Prediction results are successfully exported to {save_dir}")
    return all_coco_preds


In [46]:
def predict_fine_sliced_images(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")
    total_prediction_time = 0.0

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        all_object_predictions = []
        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                # Get initial predictions from your detection model
                time_start = time.time()
                # Base prediction on the sub-image
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))
                
                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)
                
                slice_params = get_slice_parameters(object_density, base_slice_size)

                # Add initial prediction time to the cumulative total.
                iteration_time = time_end
                
                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params

                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                
                    # Add sliced prediction time to the current iteration's total.
                    iteration_time += time_end_slice
                
                    preds = sliced_pred.object_prediction_list
                    total_prediction_count += len(preds)
                    print("Sliced Level Prediction Count 1: ", len(preds))
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    preds = base_pred.object_prediction_list
                    total_prediction_count += len(preds)
                    print("Sliced Level Prediction Count 2: ", len(preds))

                # Offset predictions to original image coordinates
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)
    print("________________________________________________")
    print("Count Prediction:",len(all_object_predictions))
    # Apply NMS to merged predictions
    merged_preds = nms_merge(all_object_predictions, iou_threshold=0.3)

    # Visualization for the current slice
    slice_filename = f"{filename_wo_ext}_slice_r{row}_c{col}"
    visualize_object_predictions(
        image=np.ascontiguousarray(sub_img),
        object_prediction_list=merged_preds,
        rect_th=vis_params["bbox_thickness"],
        text_size=vis_params["text_size"],
        text_th=vis_params["text_thickness"],
        hide_labels=vis_params["hide_labels"],
        hide_conf=vis_params["hide_conf"],
        output_dir=save_dir,
        file_name=slice_filename,
        export_format=vis_params["format"]
    )
    # Update the overall total prediction time
    total_prediction_time += iteration_time
    
    print(f"\n Completed {len(image_files)} images.")
    print("Total Prediction Count: ", (total_prediction_count))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [62]:
def predict_fine_sliced_images_correct(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")
    total_prediction_time = 0.0

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                # Get initial predictions from your detection model
                time_start = time.time()
                # Base prediction on the sub-image
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))
                
                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)
                
                slice_params = get_slice_parameters(object_density, base_slice_size)

                # Add initial prediction time to the cumulative total.
                iteration_time = time_end
                
                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params

                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                
                    # Add sliced prediction time to the current iteration's total.
                    iteration_time += time_end_slice
                
                    preds = sliced_pred.object_prediction_list
                    total_prediction_count += len(preds)
                    print("Sliced Level Prediction Count 1: ", len(preds))
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    preds = base_pred.object_prediction_list
                    total_prediction_count += len(preds)
                    print("Sliced Level Prediction Count 2: ", len(preds))

                # Offset predictions to original image coordinates
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)
                # Update the overall total prediction time
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Apply NMS to merged predictions
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.3)
        print("Merge Count Prediction:",len(merged_preds))
                
        # Visualization for the current slice
        slice_filename = f"{filename_wo_ext}_slice_r{row}_c{col}"
        visualize_object_predictions(
            image=np.ascontiguousarray(sub_img),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=slice_filename,
            export_format=vis_params["format"]
        )

    
    print(f"\n✅ Completed {len(image_files)} images.")
    print("Total Prediction Count: ", (total_prediction_count))
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [6]:
# Function to get image details by image_id
def get_image_id(coco_data, image_name):
    for image in coco_data["images"]:
        file_name = Path(image['file_name']).stem
        if file_name == image_name:
            return image['id']
    return None

In [7]:
def adaptive_filter_predictions(predictions: torch.tensor, conf_threshold: float = 0.3, min_area: float = 1024,
                      iou_min: float =0.4, iou_max: float = 0.8, area_threshold: int = 2020):
    """
    Filters out predictions with confidence scores below conf_threshold or with an area smaller than min_area.
    Args:
        predictions: tensor of shape [num_boxes, 6] where column 4 is the confidence.
        conf_threshold: Minimum confidence score required.
        min_area: Minimum area (width * height) required.
    Returns:
        Filtered predictions tensor.
    """
    scores = predictions[:, 4]
    print("Confidence Scores: ", conf_threshold)
    print("Min area Threshold: ", min_area)
    
    conf_mask = scores >= conf_threshold

    x1 = predictions[:, 0]
    y1 = predictions[:, 1]
    x2 = predictions[:, 2]
    y2 = predictions[:, 3]
    areas = (x2 - x1) * (y2 - y1)

    #area_mask = areas >= min_area

    #valid_mask = conf_mask
    valid_mask = conf_mask #& area_mask
    #valid_mask = area_mask
    filtered_areas = areas[valid_mask]

    adaptive_iou =  torch.where(filtered_areas < area_threshold, iou_max, iou_min)
    
    return predictions[valid_mask], adaptive_iou

In [8]:
def truncated_nms(
    predictions: torch.Tensor,
    adaptive_iou:torch.float32, # Truncation IoU threshold for keeping boxes
    match_metric: str = "IOU",
    IOIt: float = 0.5,  # Inside IoU threshold (IOIt: 0.3–1.0; IOOt: 0–0.3)
    IOOt: float = 0.3   # Outside IoU threshold (Upper bound of IOOt is lower bound of IOIt)
):
    """
    Apply truncated non-maximum suppression to avoid detecting too many
    overlapping bounding boxes for a given object, with added truncation logic.

    Args:
        predictions (tensor): The location preds for the image along with the class scores, Shape: [num_boxes, 5].
        match_metric (str): IOU or IOS (Intersection over Area or Intersection over Union)
        match_threshold (float): The overlap threshold for match metric.
        IOUt (float): Intersection over Union threshold for truncation (threshold to keep boxes)
        IOIt (float): Inside Intersection over Union threshold (threshold to keep inside box)
        IOOt (float): Outside Intersection over Union threshold (threshold for outside box)
    
    Returns:
        List: A list of filtered indexes
    """

    print("Adaptive Filtered Prediction: ", len(predictions))
    
    # Extract coordinates for every prediction box present in P
    x1 = predictions[:, 0]
    y1 = predictions[:, 1]
    x2 = predictions[:, 2]
    y2 = predictions[:, 3]

    # Extract the confidence scores
    scores = predictions[:, 4]

    # Calculate area of every box
    areas = (x2 - x1) * (y2 - y1)

    # Sort the prediction boxes in P according to their confidence scores
    order = scores.argsort()

    # Initialize an empty list for filtered prediction boxes
    keep = []

    while len(order) > 0:
        # Extract the index of the prediction with the highest score (S)
        idx = order[-1]

        # Push S in filtered predictions list
        keep.append(idx.tolist())

        # Remove S from P
        order = order[:-1]

        # Sanity check
        if len(order) == 0:
            break

        # Select coordinates of remaining boxes according to the indices in order
        xx1 = torch.index_select(x1, dim=0, index=order)
        xx2 = torch.index_select(x2, dim=0, index=order)
        yy1 = torch.index_select(y1, dim=0, index=order)
        yy2 = torch.index_select(y2, dim=0, index=order)

        # Find the coordinates of the intersection boxes
        xx1 = torch.max(xx1, x1[idx])
        yy1 = torch.max(yy1, y1[idx])
        xx2 = torch.min(xx2, x2[idx])
        yy2 = torch.min(yy2, y2[idx])

        # Find height and width of the intersection boxes
        w = xx2 - xx1
        h = yy2 - yy1

        # Take max with 0.0 to avoid negative width and height
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)

        # Find the intersection area
        inter = w * h

        # Find the areas of the remaining boxes according to the indices in order
        rem_areas = torch.index_select(areas, dim=0, index=order)

        # Calculate the match metric value (IoU or IoS)
        if match_metric == "IOU":
            # Find the union of every prediction T in P with the prediction S
            union = (rem_areas - inter) + areas[idx]
            match_metric_value = inter / union
        elif match_metric == "IOS":
            # Find the smaller area of every prediction T in P with the prediction S
            smaller = torch.min(rem_areas, areas[idx])
            match_metric_value = inter / smaller
        else:
            raise ValueError("Invalid match_metric. Choose either 'IOU' or 'IOS'.")

        # Add the condition for Truncated NMS:
        # - Keep boxes with IoU below the truncation threshold (IOUt)
        # - Handle inside (IOIt) and outside (IOOt) intersection thresholds
        iou_thresholds = torch.index_select(adaptive_iou, 0, order)
        
        mask = (match_metric_value < iou_thresholds)  # Keep boxes with IoU below threshold

        # Apply the truncated NMS conditions
        for i, m in enumerate(mask):
            # Condition for truncated NMS (condition 1 and 2 from original pseudocode)
            iou = match_metric_value[i]
            if iou > IOIt and iou <= IOOt:  # Inside box condition
                mask[i] = False  # Remove box if condition is met
            elif iou <= IOOt and iou > IOIt:  # Outside box condition
                mask[i] = True  # Keep box if condition is met

        # Filter out the boxes based on the updated mask
        order = order[mask]

    print("Final Bounding Box Count (Truncated NMS): ", len(keep))
    return keep

In [33]:
def predict_fine_sliced_images_refactored(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []
    keep = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)
        img_id = get_image_id(data, filename_wo_ext)
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    preds = sliced_pred.object_prediction_list
                    coco_prediction = sliced_pred.to_coco_predictions(image_id=img_id)
                    for idx, predict in enumerate(coco_prediction):
                        if coco_prediction[idx]["bbox"]:
                            all_coco_preds.append(predict)
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    preds = base_pred.object_prediction_list
                    print("Base Prediction:" , base_pred)
                    coco_prediction = base_pred.to_coco_predictions(image_id=img_id)
                    for idx, predict in enumerate(coco_prediction):
                        if coco_prediction[idx]["bbox"]:
                            all_coco_preds.append(predict)
                    
                for pred in preds:
                    print
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        #merged_preds = nms_merge(all_object_predictions, iou_threshold=0.1)
        #print("Merged Prediction Count", len(merged_preds))

        #all_coco_preds_merge = []
        # COCO conversion
        #coco_preds = [p.to_coco_annotations(image_id=img_id) for p in merged_preds]
        #all_coco_preds_merge.append(coco_preds)


        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=all_object_predictions,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\n✅ Completed {len(image_files)} images.")
    print("Total Prediction Count: ", grand_total_predictions)
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [92]:
def merged_preds_to_coco(preds, image_id):
    coco_predictions = []
    for pred in preds:
        try:
            coco = pred.to_coco_prediction(image_id=image_id)
            coco_dict = {
                "image_id": coco.image_id,
                "bbox": coco.bbox,
                "score": coco.score,
                "category_id": coco.category_id,
                "segmentation": coco.segmentation,
                "iscrowd": coco.iscrowd,
                "area": coco.area,
            }
            # Optionally include category_name (not used in COCO eval)
            # coco_dict["category_name"] = coco.category_name

            # Only add if bbox is valid (non-empty)
            if coco_dict["bbox"]:
                coco_predictions.append(coco_dict)
        except Exception as e:
            print(f"Failed to convert to COCO dict: {e}")
    return coco_predictions


In [89]:
# Helper function to convert merged predictions to COCO format
def merged_preds_to_coco_bk(preds, image_id):
    coco_predictions = []
    for pred in preds:
        coco_dicts = pred.to_coco_prediction(image_id=image_id)
        print("__________________________________________________")
        print(coco_dicts)
        for coco in coco_dicts:
             print(coco)
             if "bbox" in coco and coco["bbox"]:
                print("true")
                coco_predictions.append(coco)
    return coco_predictions


In [107]:
def predict_fine_sliced_images_refactored_2(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)
        
    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")
    total_prediction_time = 0.0
    grand_total_predictions = 0
    all_coco_preds = []

    for filename in image_files:
        image_path = os.path.join(input_folder, filename)
        image_pil = Image.open(image_path).convert("RGB")
        image_np = np.array(image_pil)
        image_h, image_w = image_np.shape[:2]
        filename_wo_ext = Path(filename).stem
        total_prediction_count = 0
        print("*****************************************")
        print("File Name", filename_wo_ext)

        img_id = get_image_id(data, filename_wo_ext) if dataset_json_path else None
        all_object_predictions = []

        # Split image into 2x2 grid
        grid_h, grid_w = image_h // 2, image_w // 2

        for row in range(2):
            for col in range(2):
                x1, y1 = col * grid_w, row * grid_h
                x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
                sub_img = image_pil.crop((x1, y1, x2, y2))
                print("Cropped Image:", x1, y1, x2, y2)

                time_start = time.time()
                base_pred = get_prediction(sub_img, detection_model)
                time_end = time.time() - time_start
                print("Initial Prediction time is: {:.2f} ms".format(time_end * 1000))

                object_density = len(base_pred.object_prediction_list)
                print("Object Density:", object_density)

                slice_params = get_slice_parameters(object_density, base_slice_size)

                iteration_time = time_end

                if slice_params:
                    slice_width, slice_height, overlap_w, overlap_h = slice_params
                    print("********* Slice Parameters ***********")
                    print("Slice Width: ", slice_width)
                    print("Slice Height: ", slice_height)
                    print("Overlap Width Ratio: ", overlap_w)
                    print("Overlap Height Ratio: ", overlap_h)

                    time_start_slice = time.time()
                    sliced_pred = get_sliced_prediction(
                        sub_img,
                        detection_model,
                        slice_height=slice_height,
                        slice_width=slice_width,
                        overlap_height_ratio=overlap_h,
                        overlap_width_ratio=overlap_w,
                        postprocess_type="OptNMS",
                        postprocess_match_metric="IOU",
                        postprocess_match_threshold=0.3,
                        postprocess_min_area=16,
                        verbose=0
                    )
                    time_end_slice = time.time() - time_start_slice
                    print("Sliced Prediction time is: {:.2f} ms".format(time_end_slice * 1000))
                    
                    iteration_time += time_end_slice
                    preds = sliced_pred.object_prediction_list
                else:
                    print("Prediction time is: {:.2f} ms".format(time_end * 1000))
                    preds = base_pred.object_prediction_list

                # Offset bounding boxes back to original image space
                for pred in preds:
                    pred.bbox.minx += x1
                    pred.bbox.maxx += x1
                    pred.bbox.miny += y1
                    pred.bbox.maxy += y1
                    all_object_predictions.append(pred)

                total_prediction_count += len(preds)
                total_prediction_time += iteration_time

        print("________________________________________________")

        # Merge all predictions for image
        merged_preds = nms_merge(all_object_predictions, iou_threshold=0.5)
        print("Merge Count Prediction:", len(merged_preds))

        # Convert merged predictions to COCO format
        if dataset_json_path:
            coco_preds = merged_preds_to_coco(merged_preds, img_id)
            all_coco_preds.extend(coco_preds)
            print("COCO formatted Prediction Count:", len(all_coco_preds))
            
        # Visualization
        visualize_object_predictions(
            image=np.ascontiguousarray(image_pil),
            object_prediction_list=merged_preds,
            rect_th=vis_params["bbox_thickness"],
            text_size=vis_params["text_size"],
            text_th=vis_params["text_thickness"],
            hide_labels=vis_params["hide_labels"],
            hide_conf=vis_params["hide_conf"],
            output_dir=save_dir,
            file_name=filename_wo_ext,
            export_format=vis_params["format"]
        )

        grand_total_predictions += total_prediction_count

    if dataset_json_path:
        save_path = str(save_dir / "result.json")
        save_json(all_coco_preds, save_path)
   
    print(f"\n Completed {len(image_files)} images.")
    print("Total Prediction Count: ", grand_total_predictions)
    print("Total Prediction time for all images is: {:.2f} ms".format(total_prediction_time * 1000))
    print(f"Prediction results are successfully exported to {save_dir}")


In [108]:

#latest - OptNMS Take6
source_folder = './single_test/images'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.59 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.59 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.39 ms
Object Density: 6
Prediction time is: 7.39 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.37 ms
Obje

In [109]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp320/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.422
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.754
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.493
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.750
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.479
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [100]:

#latest - OptNMS Take6
source_folder = './single_test/images_bk'
json_path = "./subset_visdrone_test_990.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000006_06773_d_0000018
Cropped Image: 0 0 680 382
Initial Prediction time is: 20.31 ms
Object Density: 15
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  29
Final Bounding Box Count (OptNMS): 16
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  6
Final Bounding Box Count (OptNMS): 5
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.93 ms
Cropped Image: 680 0 1360 382
Initial Prediction time is: 7.36 ms
Object Density: 6
Prediction time is: 7.36 ms
Cropped Image: 0 382 680 764
Initial Prediction time is: 7.32 ms
Obj

In [101]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './sliced_predictions/exp317/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.383
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.694
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.434
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.976
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.664
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.574
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.435
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.424
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [110]:
#Threshold IOU = 0.5
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_visdrone_test_990.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.5,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000006_06773_d_0000018
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.65it/s]

Original Prediction Count 143
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 491.99 ms
Prediction results are successfully exported to runs/predict/exp235
Model loaded in 0.032637596130371094 seconds.
Slicing performed in 0.0013730525970458984 seconds.
Prediction performed in 0.4919910430908203 seconds.
Exporting performed in 0.049646854400634766 seconds.


In [111]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './runs/predict/exp235/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.958
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.741
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [102]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_visdrone_test_990.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000006_06773_d_0000018
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.66it/s]

Original Prediction Count 143
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  29
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  12
Final Bounding Box Count (NMS):  6
Final Bounding Box Count (NMS):  1
Prediction time is: 488.65 ms
Prediction results are successfully exported to runs/predict/exp234
Model loaded in 0.10470438003540039 seconds.
Slicing performed in 0.0013213157653808594 seconds.
Prediction performed in 0.4886488914489746 seconds.
Exporting performed in 0.0502934455871582 seconds.


In [103]:
#USING Adaptive-Optimized-NMS-IoU METHOD (NMS)
!sahi coco evaluate --dataset_json_path './subset_visdrone_test_990.json' --result_json_path './runs/predict/exp234/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.391
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.704
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.426
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.958
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.741
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = 0.287
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.402
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.468
 Average Precision  (AP) @[ IoU=0.50:0.95 | a

In [97]:
#latest - OptNMS Take6
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
result = predict_fine_sliced_images_refactored_2(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 9.78 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 61.60 ms
Cropped Image: 680 0 1360 382
Initial Pre

In [98]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp316/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [95]:
#USING Adaptive-Optimized-NMS-IoU METHOD (TruncatedNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp315/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.120
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.035
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.129
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.211
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.044
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.119
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [12]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp272/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [91]:
#latest - OptNMS Take5
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images_refactored(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.67 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 

In [92]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './sliced_predictions/exp270/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.019
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.006
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.031
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.013
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.009
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [36]:
result_predict_nms = predict(source='./single_test',
                         dataset_json_path = './subset_vis_test_data_1162.json',
                         model_type = 'ultralytics',
                         model_path = 'models/yolov8/last.pt',
                         slice_height = 256,
                         slice_width = 256,
                         overlap_height_ratio = 0.5,
                         overlap_width_ratio = 0.5,
                         postprocess_type = "NMS",
                         postprocess_min_area = 16,
                         postprocess_conf_threshold = 0.3,                  
                         verbose = 2
                        )

POST PROCESSING: NMS
indexing coco dataset annotations...


Performing inference on images:   0%|          | 0/1 [00:00<?, ?it/s]

Image Name: 0000011_05068_d_0000008
Image Size:  (1360, 765)
Sliced Boxes Count: 50
POST PROCESS:  NMS
Performing prediction on 50 slices.


Performing inference on images: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]

Original Prediction Count 271
Final Bounding Box Count (NMS):  44
Final Bounding Box Count (NMS):  7
Final Bounding Box Count (NMS):  30
Final Bounding Box Count (NMS):  22
Final Bounding Box Count (NMS):  5
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS):  11
Prediction time is: 510.31 ms
Prediction results are successfully exported to runs/predict/exp233
Model loaded in 0.18946099281311035 seconds.
Slicing performed in 0.001336812973022461 seconds.
Prediction performed in 0.5103139877319336 seconds.
Exporting performed in 0.04958391189575195 seconds.


In [37]:
#USING Adaptive-Optimized-NMS-IoU METHOD (OptNMS)
!sahi coco evaluate --dataset_json_path './subset_vis_test_data_1162.json' --result_json_path './runs/predict/exp233/result.json'

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!

Evaluating bbox...
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.01s).
Accumulating evaluation results...
DONE (t=0.00s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=500 ] = 0.080
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=500 ] = 0.189
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=500 ] = 0.038
 Average Precision  (AP) @[ IoU=0.50      | area= small | maxDets=500 ] = 0.169
 Average Precision  (AP) @[ IoU=0.50      | area=medium | maxDets=500 ] = 0.429
 Average Precision  (AP) @[ IoU=0.50      | area= large | maxDets=500 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=500 ] = 0.052
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=500 ] = 0.252
 Average Precision  (AP) @[ IoU=0.50:0.95 | 

In [23]:
#latest - OptNMS Take4
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,            # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size              # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Initial Prediction time is: 8.60 ms
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Prediction time is: 

In [15]:
#latest - OptNMS Take3
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 256

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  256
Slice Height:  256
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 6
POST PROCESS:  OptNMS
Original Prediction Count 32
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  24
Final Bounding Box Count (OptNMS): 18
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 2
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count 1:  25
Cropped Image: 680 0 1360 

In [14]:
#latest - OptNMS Take2
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Original Prediction Count 37
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  32
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count 1:  24
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height

In [10]:
#latest - NMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  NMS
Original Prediction Count 37
Final Bounding Box Count (NMS):  15
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Final Bounding Box Count (NMS):  1
Sliced Level Prediction Count:  19
Total Prediction Count:  19
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  NMS
Original Prediction Count 41
Final Bounding Box Count (NMS):  10
Final Bounding Box Count (NMS):  4
Final Bounding Box Count (NMS):  2
Final Bounding Box Count (NMS): 

In [8]:
#latest - OptNMS
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)


🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Original Prediction Count 37
Intial Min Area:  16
Confidence Scores:  0.3
Min area Threshold:  16
Adaptive Filtered Prediction:  32
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count:  24
Total Prediction Count:  24
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width R

In [43]:
source_folder = './single_test/images'
json_path = "./subset_vis_test_data_1162.json"
slice_size = 512

# Run fine-sliced adaptive prediction
predict_fine_sliced_images(
    input_folder=source_folder,             # Folder with input images
    dataset_json_path=json_path,  # Optional: COCO JSON for ID mapping (or use None)
    detection_model=detection_model,
    base_slice_size=slice_size                        # You can try 256 or 640 depending on image/object scale
)



🚀 Running fine slicing prediction on 1 images...
*****************************************
File Name 0000011_05068_d_0000008
Cropped Image: 0 0 680 382
Object Density: 13
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width Ratio:  0.15
Overlap Height Ratio:  0.15
Sliced Boxes Count: 2
POST PROCESS:  OptNMS
Original Prediction Count 37
Intial Min Area:  32
Confidence Scores:  0.3
Min area Threshold:  32
Adaptive Filtered Prediction:  32
Final Bounding Box Count (OptNMS): 20
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  2
Final Bounding Box Count (OptNMS): 1
Adaptive Filtered Prediction:  1
Final Bounding Box Count (OptNMS): 1
Sliced Level Prediction Count:  24
Total Prediction Count:  24
Cropped Image: 680 0 1360 382
Object Density: 20
********* Slice Parameters ***********
Slice Width:  512
Slice Height:  512
Overlap Width R

In [ ]:
def process_image_fine_slicing(args):
    filename, input_folder, detection_model, base_slice_size, data, save_dir, vis_params = args
    image_path = os.path.join(input_folder, filename)
    image_pil = Image.open(image_path).convert("RGB")
    image_np = np.array(image_pil)
    image_h, image_w = image_np.shape[:2]
    filename_wo_ext = Path(filename).stem
    img_id = next((img["id"] for img in data.get("images", []) if img["file_name"].startswith(filename_wo_ext)), None)

    print("*****************************************")
    print("File Name", filename_wo_ext)
    
    all_object_predictions = []
    total_time = 0

    # Split image into 2x2 grid
    grid_h, grid_w = image_h // 2, image_w // 2

    for row in range(2):
        for col in range(2):
            x1, y1 = col * grid_w, row * grid_h
            x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
            sub_img = image_pil.crop((x1, y1, x2, y2)) #grid cell

            # Base prediction on the sub-image
            base_pred = get_prediction(sub_img, detection_model)
            object_density = len(base_pred.object_prediction_list)

            print("Object Density:", object_density)
            
            slice_params = get_slice_parameters(object_density, base_slice_size)

            if slice_params:
                slice_width, slice_height, overlap_w, overlap_h = slice_params

                print("********* Slice Parameters ***********")
                print("Slice Width: ", slice_width)
                print("Slice Height: ", slice_height)
                print("Overlap Width Ratio: ", overlap_w)
                print("Overlap Height Ratio: ", overlap_h)
                
                sliced_pred = get_sliced_prediction(
                    sub_img,
                    detection_model,
                    slice_height=slice_height,
                    slice_width=slice_width,
                    overlap_height_ratio=overlap_h,
                    overlap_width_ratio=overlap_w,
                    postprocess_type="OptNMS",
                    postprocess_match_metric="IOU",
                    postprocess_match_threshold=0.3,
                    postprocess_min_area=32,
                    verbose=0
                )
                preds = sliced_pred.object_prediction_list
            else:
                preds = base_pred.object_prediction_list

            # Offset predictions to original image coordinates
            for pred in preds:
                pred.shift(x_offset=x1, y_offset=y1)
                all_object_predictions.append(pred)

    # Apply NMS to merged predictions
    merged_preds = nms_merge(all_object_predictions, iou_threshold=0.3)

    # Visualization
    visualize_object_predictions(
        image=np.ascontiguousarray(image_pil),
        object_prediction_list=merged_preds,
        rect_th=vis_params["bbox_thickness"],
        text_size=vis_params["text_size"],
        text_th=vis_params["text_thickness"],
        hide_labels=vis_params["hide_labels"],
        hide_conf=vis_params["hide_conf"],
        output_dir=save_dir,
        file_name=filename_wo_ext,
        export_format=vis_params["format"]
    )

    # COCO conversion
    coco_preds = [p.to_coco_dict(image_id=img_id) for p in merged_preds]
    return coco_preds

In [ ]:
import os
import time
import json
import numpy as np
from PIL import Image
from pathlib import Path
from multiprocessing import Pool, cpu_count
from sahi.prediction import get_prediction, get_sliced_prediction
from sahi.utils.file import save_json, increment_path
from sahi.utils.cv import read_image
from sahi.utils.merging import merge_object_prediction_list
from sahi.visualization import visualize_object_predictions


def get_dynamic_slice_parameters(object_density, base_slice_size):
    if object_density > 30:
        return base_slice_size // 2, base_slice_size // 2, 0.4, 0.4
    elif object_density > 15:
        return base_slice_size // 2, base_slice_size // 2, 0.3, 0.3
    elif object_density > 5:
        return base_slice_size, base_slice_size, 0.2, 0.2
    else:
        return None


def process_image_fine_slicing(args):
    filename, input_folder, detection_model, base_slice_size, data, save_dir, vis_params = args
    image_path = os.path.join(input_folder, filename)
    image_pil = Image.open(image_path).convert("RGB")
    image_np = np.array(image_pil)
    image_h, image_w = image_np.shape[:2]
    filename_wo_ext = Path(filename).stem
    img_id = next((img["id"] for img in data.get("images", []) if img["file_name"].startswith(filename_wo_ext)), None)

    all_object_predictions = []
    total_time = 0

    # Split image into 2x2 grid
    grid_h, grid_w = image_h // 2, image_w // 2

    for row in range(2):
        for col in range(2):
            x1, y1 = col * grid_w, row * grid_h
            x2, y2 = min(x1 + grid_w, image_w), min(y1 + grid_h, image_h)
            sub_img = image_pil.crop((x1, y1, x2, y2))

            # Base prediction on the sub-image
            base_pred = get_prediction(sub_img, detection_model)
            object_density = len(base_pred.object_prediction_list)

            slice_params = get_dynamic_slice_parameters(object_density, base_slice_size)

            if slice_params:
                slice_width, slice_height, overlap_w, overlap_h = slice_params
                sliced_pred = get_sliced_prediction(
                    sub_img,
                    detection_model,
                    slice_height=slice_height,
                    slice_width=slice_width,
                    overlap_height_ratio=overlap_h,
                    overlap_width_ratio=overlap_w,
                    postprocess_type="NMS",
                    postprocess_match_metric="IOU",
                    postprocess_match_threshold=0.5,
                    postprocess_min_area=32,
                    verbose=0
                )
                preds = sliced_pred.object_prediction_list
            else:
                preds = base_pred.object_prediction_list

            # Offset predictions to original image coordinates
            for pred in preds:
                pred.shift(x_offset=x1, y_offset=y1)
                all_object_predictions.append(pred)

    # Merge predictions
    merged_preds = merge_object_prediction_list(all_object_predictions)

    # Visualization
    visualize_object_predictions(
        image=np.ascontiguousarray(image_pil),
        object_prediction_list=merged_preds,
        rect_th=vis_params["bbox_thickness"],
        text_size=vis_params["text_size"],
        text_th=vis_params["text_thickness"],
        hide_labels=vis_params["hide_labels"],
        hide_conf=vis_params["hide_conf"],
        output_dir=save_dir,
        file_name=filename_wo_ext,
        export_format=vis_params["format"]
    )

    # COCO conversion
    coco_preds = [p.to_coco_dict(image_id=img_id) for p in merged_preds]
    return coco_preds


def predict_fine_sliced_images(input_folder, dataset_json_path, detection_model, base_slice_size=512):
    name = "exp"
    save_dir = Path(increment_path(Path("sliced_predictions") / name, exist_ok=False))
    os.makedirs(save_dir, exist_ok=True)

    data = {}
    if dataset_json_path:
        with open(dataset_json_path, "r") as file:
            data = json.load(file)

    vis_params = {
        "bbox_thickness": 2,
        "text_size": 0.5,
        "text_thickness": 1,
        "hide_labels": False,
        "hide_conf": False,
        "format": "png"
    }

    image_files = [
        f for f in os.listdir(input_folder)
        if f.lower().endswith((".jpg", ".jpeg", ".png"))
    ]

    tasks = [
        (filename, input_folder, detection_model, base_slice_size, data, save_dir, vis_params)
        for filename in image_files
    ]

    print(f"\n🚀 Running fine slicing prediction on {len(image_files)} images...")

    all_coco_preds = []
    with Pool(processes=min(cpu_count(), 4)) as pool:
        for preds in pool.imap_unordered(process_image_fine_slicing, tasks):
            all_coco_preds.extend(preds)

    if dataset_json_path:
        save_json(all_coco_preds, str(save_dir / "result.json"))
        print(f"\n📦 Saved COCO results to {save_dir / 'result.json'}")

    print(f"\n✅ Completed {len(image_files)} images.")
    return all_coco_preds
